In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import shapely.geometry

from pymedphys._dicom.create import dicom_dataset_from_dict
from pymedphys._dicom.coords import xyz_axes_from_dataset
from pymedphys._dicom.dose import get_dose_grid_structure_mask

In [ ]:
dx = 0.1
dy = 0.2

x0 = -1
y0 = -1

x_grid = np.arange(x0, 2, dx)
y_grid = np.arange(y0, 3, dy)

xx, yy = np.meshgrid(x_grid, y_grid)

In [ ]:
xx_flat, yy_flat = xx.ravel(), yy.ravel()
points = shapely.geometry.MultiPoint(list(zip(xx_flat, yy_flat)))
points

In [ ]:
contour_x = [0, 0, 1, 1]
contour_y = [0, 2, 2, 0]

contours = np.vstack([contour_x, contour_y]).T
contours

In [ ]:
rectangle = shapely.geometry.Polygon(contours)
rectangle

In [ ]:
points_within_rectangle = points.intersection(rectangle)
points_within_rectangle

In [ ]:
assert points_within_rectangle.within(rectangle)

In [ ]:
mask = np.zeros_like(xx).astype('bool')

for point in points_within_rectangle:
    coord = point.coords.xy
    mask[np.logical_and(xx==coord[0], yy==coord[1])] = True
    
plt.scatter(xx, yy, c=mask)
plt.plot(contour_x, contour_y, lw=2)

In [ ]:
buffered_rectangle = rectangle.buffer(0.001)
buffered_rectangle

In [ ]:
points_within_buffered_rectangle = points.intersection(buffered_rectangle)

buffered_mask = np.zeros_like(xx).astype('bool')

for point in points_within_buffered_rectangle:
    coord = point.coords.xy
    buffered_mask[np.logical_and(xx==coord[0], yy==coord[1])] = True
    
plt.scatter(xx, yy, c=buffered_mask)
plt.plot(contour_x, contour_y, lw=2)

In [ ]:
contour_z = [0, 1, 2]

contour_data = []

for z in contour_z:
    concatenated_contours = []
    for x, y in zip(contour_x, contour_y):
        concatenated_contours.append(x)
        concatenated_contours.append(y)
        concatenated_contours.append(z)
        
    contour_data.append(concatenated_contours)
    

contour_data

In [ ]:
contour_name = 'rectangle'

structure_dataset = dicom_dataset_from_dict(
    {
        'StructureSetROISequence': [
            {
                'ROINumber': 1,
                'ROIName': contour_name
            }
        ],
        'ROIContourSequence': [
            {
                'ReferencedROINumber': 1,
                'ContourSequence': [
                    {
                        'ContourData': contour_data[0],
                    },
                    {
                        'ContourData': contour_data[1],
                    },
                    {
                        'ContourData': contour_data[2],
                    }
                ]
            }
        ]
    }
)

structure_dataset

In [ ]:
dose_dataset = dicom_dataset_from_dict(
    {
        'Columns': len(x_grid),
        'Rows': len(y_grid),
        'PixelSpacing': [dx, dy],
        'ImagePositionPatient': [x0, y0, contour_z[0]],
        'ImageOrientationPatient': [1, 0, 0, 0, 1, 0],  # HFS
        'GridFrameOffsetVector': contour_z
    }
)

dose_dataset

In [ ]:
x_dicom, y_dicom, z_dicom = xyz_axes_from_dataset(dose_dataset)

In [ ]:
assert np.allclose(x_dicom, x_grid)

In [ ]:
assert np.allclose(y_dicom, y_grid)

In [ ]:
assert np.allclose(z_dicom, contour_z)

In [ ]:
dicom_mask = get_dose_grid_structure_mask(contour_name, structure_dataset, dose_dataset)

In [ ]:
plt.scatter(xx, yy, c=dicom_mask[0,:,:])
plt.plot(contour_x, contour_y, lw=2)

In [ ]:
plt.scatter(xx, yy, c=dicom_mask[1,:,:])
plt.plot(contour_x, contour_y, lw=2)

In [ ]:
plt.scatter(xx, yy, c=dicom_mask[2,:,:])
plt.plot(contour_x, contour_y, lw=2)